In [1]:
import pandas as pd
import pandera as pa

In [2]:
valores_ausentes = ['**', '###!', '####', '****', '*****', 'NULL']
df = pd.read_csv('ocorrencia_2010_2020.csv', sep=';', parse_dates=['ocorrencia_dia'], dayfirst=True, na_values=valores_ausentes)
df.head(10)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,NaN,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
5,39807,39807,INCIDENTE,SALVADOR,BA,NaN,2010-01-06,17:53:00,0
6,40215,40215,INCIDENTE,COARI,AM,SBUY,2010-01-07,18:40:00,0
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
8,39156,39156,INCIDENTE GRAVE,CASCAVEL,PR,SBCA,2010-01-10,23:15:00,2
9,39711,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,NaN,2010-01-10,20:00:00,0


In [3]:
schema = pa.DataFrameSchema(
    columns = {
        "codigo":pa.Column(pa.Int, required=False),
        "codigo_ocorrencia":pa.Column(pa.Int),
        "codigo_ocorrencia2":pa.Column(pa.Int),
        "ocorrencia_classificacao":pa.Column(pa.String),
        "ocorrencia_cidade":pa.Column(pa.String),
        "ocorrencia_uf":pa.Column(pa.String, pa.Check.str_length(2,2), nullable=True),
        "ocorrencia_aerodromo":pa.Column(pa.String, nullable=True),
        "ocorrencia_dia":pa.Column(pa.DateTime),
        "ocorrencia_hora":pa.Column(pa.String, pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5][0-9])(:[0-5][0-9])?$'), nullable=True),
        "total_recomendacoes":pa.Column(pa.Int)
    }
)

In [4]:
schema.validate(df)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,NaN,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...,...
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,NaN,2020-12-30,18:30:00,0
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0
5750,79756,79756,INCIDENTE GRAVE,VICENTINA,MS,NaN,2020-12-31,09:00:00,0


In [5]:
df.dtypes

codigo_ocorrencia                    int64
codigo_ocorrencia2                   int64
ocorrencia_classificacao            object
ocorrencia_cidade                   object
ocorrencia_uf                       object
ocorrencia_aerodromo                object
ocorrencia_dia              datetime64[ns]
ocorrencia_hora                     object
total_recomendacoes                  int64
dtype: object

In [6]:
# Ocorrência com mais de 10 recomendações
filtro = df.total_recomendacoes > 10
df.loc[filtro, ['ocorrencia_cidade', 'total_recomendacoes']]

,ocorrencia_cidade,total_recomendacoes
63,SÃO PAULO,11
326,RIO DE JANEIRO,19
530,SÃO PAULO,11
727,ITÁPOLIS,11
872,RECIFE,23
884,SÃO JOSÉ DOS PINHAIS,11
1062,UBERLÂNDIA,20
1972,BRASÍLIA,11
2795,SANTOS,13
3930,VITÓRIA,12


In [7]:
# Ocorrência de classificação INCIDENTE GRAVE e com mais de 10 recomendações
filtro1 = df.ocorrencia_classificacao.isin(['INCIDENTE GRAVE', 'INCIDENTE'])
df.loc[filtro1]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
5,39807,39807,INCIDENTE,SALVADOR,BA,NaN,2010-01-06,17:53:00,0
...,...,...,...,...,...,...,...,...,...
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,NaN,2020-12-30,18:30:00,0
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0
5750,79756,79756,INCIDENTE GRAVE,VICENTINA,MS,NaN,2020-12-31,09:00:00,0


In [8]:
# Ocorrência cuja cidade começam com C
filtro1 = df.ocorrencia_cidade.str[0] == 'C'
df.loc[filtro1]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
6,40215,40215,INCIDENTE,COARI,AM,SBUY,2010-01-07,18:40:00,0
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
8,39156,39156,INCIDENTE GRAVE,CASCAVEL,PR,SBCA,2010-01-10,23:15:00,2
15,39315,39315,ACIDENTE,CANUTAMA,AM,NaN,2010-01-15,21:22:00,0
20,40310,40310,INCIDENTE,CRUZEIRO DO SUL,AC,SBCZ,2010-01-18,19:32:00,0
...,...,...,...,...,...,...,...,...,...
5661,79649,79649,INCIDENTE,CAMPINAS,SP,SBKP,2020-11-04,22:39:00,0
5663,79652,79652,INCIDENTE,CONFINS,MG,SBCF,2020-11-06,14:20:00,0
5680,79683,79683,INCIDENTE,CRUZEIRO DO SUL,AC,SBCZ,2020-11-16,22:00:00,0
5696,79692,79692,ACIDENTE,CATANDUVA,SP,SDCD,2020-11-28,12:50:00,0


In [9]:
# Ocorrência cuja cidade terminam com MA
filtro1 = df.ocorrencia_cidade.str[-2:] == 'MA'
df.loc[filtro1]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
15,39315,39315,ACIDENTE,CANUTAMA,AM,NaN,2010-01-15,21:22:00,0
408,43062,43062,ACIDENTE,NOVO GAMA,GO,NaN,2010-09-25,16:30:00,1
688,44198,44198,ACIDENTE,BURITAMA,SP,NaN,2011-03-19,11:18:00,0
1035,51785,51785,INCIDENTE,BURITAMA,SP,SIBX,2011-10-25,19:15:00,0
1377,45939,45939,ACIDENTE,UMUARAMA,PR,NaN,2012-05-30,19:00:00,0
1401,46180,46180,ACIDENTE,AURIFLAMA,SP,NaN,2012-06-16,19:30:00,0
2099,49113,49113,INCIDENTE GRAVE,NOVA LIMA,MG,NaN,2013-06-15,12:00:00,4
2102,48799,48799,ACIDENTE,CANUTAMA,AM,NaN,2013-06-18,21:30:00,0
2655,51347,51347,INCIDENTE,DIADEMA,SP,NaN,2014-05-04,16:00:00,0


In [10]:
# Ocorrência cuja cidade contêm MA ou AL
filtro1 = df.ocorrencia_cidade.str.contains('MA|AL')
df.loc[filtro1]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
5,39807,39807,INCIDENTE,SALVADOR,BA,NaN,2010-01-06,17:53:00,0
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
13,39507,39507,ACIDENTE,PRIMAVERA DO LESTE,MT,NaN,2010-01-11,17:00:00,2
15,39315,39315,ACIDENTE,CANUTAMA,AM,NaN,2010-01-15,21:22:00,0
17,39810,39810,INCIDENTE,SALVADOR,BA,NaN,2010-01-15,18:23:00,0
...,...,...,...,...,...,...,...,...,...
5738,79787,79787,INCIDENTE,FORTALEZA,CE,SBFZ,2020-12-23,18:30:00,0
5739,79749,79749,ACIDENTE,MARITUBA,PA,NaN,2020-12-26,12:05:00,0
5741,79754,79754,INCIDENTE GRAVE,NOVA MARINGÁ,MT,NaN,2020-12-28,13:00:00,0
5745,79755,79755,ACIDENTE,MATO RICO,PR,NaN,2020-12-29,10:30:00,0


In [11]:
# Ocorrências do ano 2015 e mês 12
filtro1 = df.ocorrencia_dia.dt.year == 2015
filtro2 = df.ocorrencia_dia.dt.month == 12
df2015_12 =df.loc[filtro1 & filtro2]


In [12]:
# Junção de data e hora
df['ocorrencia_dia_hora'] = pd.to_datetime(df.ocorrencia_dia.astype(str) + ' ' + df.ocorrencia_hora)

In [13]:
df.head()

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0,2010-01-03 12:00:00
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0,2010-01-03 11:05:00
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0,2010-01-03 03:00:00
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,NaN,2010-01-04,17:30:00,0,2010-01-04 17:30:00
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0,2010-01-05 19:25:00


In [14]:
# Agrupamento de e contagem de classificação com base no filtro
df2015_12.groupby(['ocorrencia_classificacao']).size().sort_values(ascending=False)

ocorrencia_classificacao
INCIDENTE          23
ACIDENTE           14
INCIDENTE GRAVE     2
dtype: int64

In [15]:
filtro_ano = df.ocorrencia_dia.dt.year == 2010
filtro_uf = df.ocorrencia_uf.isin(['SP', 'MG', 'ES', 'RJ'])
df_sudeste_2010 = df.loc[filtro_ano & filtro_uf]
df_sudeste_2010

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0,2010-01-03 12:00:00
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0,2010-01-03 03:00:00
9,39711,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,NaN,2010-01-10,20:00:00,0,2010-01-10 20:00:00
11,40069,40069,ACIDENTE,SÃO PAULO,SP,SBMT,2010-01-10,14:50:00,8,2010-01-10 14:50:00
16,39809,39809,INCIDENTE,SÃO PAULO,SP,NaN,2010-01-15,15:00:00,0,2010-01-15 15:00:00
...,...,...,...,...,...,...,...,...,...,...
541,43722,43722,ACIDENTE,CAJOBI,SP,NaN,2010-12-23,11:00:00,0,2010-12-23 11:00:00
544,43548,43548,INCIDENTE,SÃO PAULO,SP,SBSP,2010-12-26,22:10:00,0,2010-12-26 22:10:00
548,43724,43724,ACIDENTE,COSMÓPOLIS,SP,NaN,2010-12-29,12:00:00,0,2010-12-29 12:00:00
550,43730,43730,INCIDENTE,VITÓRIA,ES,NaN,2010-12-30,18:10:00,0,2010-12-30 18:10:00


In [16]:
# Todos as ocorrencias do Sudeste com mais de 1 recomendação
filtro = df_sudeste_2010.total_recomendacoes > 0
df_sudeste_2010.loc[filtro].groupby(['ocorrencia_cidade']).total_recomendacoes.sum()

ocorrencia_cidade
BELO HORIZONTE            2
BOITUVA                   1
BOM JESUS DO GALHO        1
BRAGANÇA PAULISTA         9
CAMANDUCAIA               9
IPERÓ                     3
ITANHAÉM                  1
JUNDIAÍ                   7
NOVO HORIZONTE            3
RESENDE                   8
RIO DE JANEIRO           25
SÃO BERNARDO DO CAMPO     2
SÃO PAULO                41
TIMÓTEO                   3
Name: total_recomendacoes, dtype: int64

In [17]:
# Todas as ocorrências das cidades somadas e separadas por mês
df_sudeste_2010.loc[filtro].groupby(['ocorrencia_cidade', df_sudeste_2010.ocorrencia_dia.dt.month]).total_recomendacoes.sum()

ocorrencia_cidade      ocorrencia_dia
BELO HORIZONTE         2                  1
                       3                  1
BOITUVA                7                  1
BOM JESUS DO GALHO     12                 1
BRAGANÇA PAULISTA      11                 9
CAMANDUCAIA            7                  9
IPERÓ                  1                  3
ITANHAÉM               9                  1
JUNDIAÍ                5                  5
                       6                  2
NOVO HORIZONTE         7                  3
RESENDE                4                  8
RIO DE JANEIRO         4                  3
                       7                  3
                       8                 19
SÃO BERNARDO DO CAMPO  1                  2
SÃO PAULO              1                  8
                       2                 14
                       6                  2
                       11                 6
                       12                11
TIMÓTEO                2              